In [77]:
from collections import Counter
from collections import namedtuple
import math
from functools import cache
import random
from tqdm.auto import tqdm
from enum import IntEnum

In [86]:
def load_word_list(n=5):
    with open('sowpods.txt') as f:
        return [l.strip().lower() for l in f if len(l.strip()) == 5]

words = load_word_list()
print(len(words))

12478


In [90]:
class ScoreState(IntEnum):
    YELLOW = 1
    GREEN = 2

ScoreInfo = namedtuple('ScoreInfo', 'letter position state')    

def compute_guess(words):
    return random.choice(words)

def compute_score(guess, secret):
    s = Counter(secret)
    g = Counter(guess)
    return sum([min(s.get(letter, 0), s.get(letter, 0)) for letter in g])    
    # scores = []
    # return sorted(scores)

def apply_conditions(words, conditions):
    for guess, score in conditions.items():
        words = [w for w in words if compute_score(w, guess) == score]
    return words

In [105]:
%%time
# Guess word
words = load_word_list()
conditions = {
}
words = apply_conditions(words, conditions)
print(compute_guess(words))

unket
CPU times: user 30.2 ms, sys: 5.78 ms, total: 36 ms
Wall time: 48 ms
